
show timeline history?

ROUTE = /timeline

ROOT = true

TODO: swap out military connections for artists or celebrities

TODO: reflect expansions of empires or ideas with bubble

TODO: where vikings has explored through history

TODO: show aggregate counts of military personel



In [ ]:
const mustache = require('mustache')


const locations = [
  { name: "New York", lat: 40.7128, lng: -74.006 },
  { name: "London", lat: 51.5074, lng: -0.1278 },
  { name: "Paris", lat: 48.8566, lng: 2.3522 },
  { name: "Tokyo", lat: 35.6895, lng: 139.6917 }
];

const occupations = JSON.parse(importer.interpret('occupations.json').code)


function showTimeline() {
  return mustache.render(importer.interpret('timeline template').code, {
    LOCATIONS: JSON.stringify(locations),
    TIMESTAMP: Date.now(),
  })
}

module.exports = showTimeline



timeline template?


In [ ]:
<!DOCTYPE html>
<html lang="en">

<head>
  <meta charset="UTF-8">
  <title>World Doily</title>
  <link rel="stylesheet" href="https://unpkg.com/leaflet/dist/leaflet.css" />
  <link rel="stylesheet" href="/timeline-styles.css?t={{{TIMESTAMP}}}">
  <script>let locations = JSON.parse('{{{LOCATIONS}}}');</script>
</head>

<body>
  <div id="map"></div>
  <div id="info">Zoom & drag to explore</div>
  <div id="category-toolbar">
    <input type="text" id="searchBox" placeholder="Search places..." />
  
    <button data-category="countries">Countries</button>
    <button data-category="cities">Cities</button>
    <button data-category="historic">Historic Places</button>
    <div id="more-menu">
      <button id="more-button">More ▼</button>
      <div id="more-dropdown" class="hidden">
        <button data-category="visited">Visited</button>
        <button data-category="battles">Battle Grounds</button>
      </div>
    </div>
  </div>
  <div id="time-slider-container">
    <input type="range" id="time-slider" min="1500" max="2025" step="1" value="2023">
    <label id="time-label">Year: 2023</label>
  </div>
  <script src="https://unpkg.com/leaflet/dist/leaflet.js"></script>
  <script src="/map-controls.js?t={{{TIMESTAMP}}}"></script>
  <script src="/timeline-script.js?t={{{TIMESTAMP}}}"></script>
</body>

</html>


timeline-styles.css?

ROUTE = /timeline-styles.css


In [ ]:
body {
  font-family: sans-serif;
  margin: 0;
  padding: 0;
  text-align: center;
}

#map {
  height: 100%;
  width: 100%;
  position: fixed;
  
}

#info {
  position: absolute;
  top: 10px;
  left: 60px;
  background: rgba(255, 255, 255, 0.9);
  padding: 10px;
  border-radius: 6px;
  box-shadow: 0 0 5px rgba(0, 0, 0, 0.2);
  z-index: 100;
  pointer-events: none;
}

#category-toolbar {
  position: absolute;
  top: 65px;
  left: 60px;
  display: flex;
  gap: 8px;
  background: rgba(255, 255, 255, 0.95);
  padding: 8px 10px;
  border-radius: 6px;
  box-shadow: 0 0 5px rgba(0, 0, 0, 0.2);
  z-index: 1000;
  flex-wrap: wrap;
}

#category-toolbar button {
  background: #f0f0f0;
  border: none;
  padding: 6px 10px;
  border-radius: 4px;
  cursor: pointer;
  font-size: 13px;
  transition: background 0.2s;
}

#category-toolbar button:hover {
  background: #ddd;
}

#more-menu {
  position: relative;
}

#more-button {
  background: #e0e0e0;
}

#more-dropdown {
  position: absolute;
  top: 100%;
  left: 0;
  margin-top: 4px;
  display: flex;
  flex-direction: column;
  background: white;
  border: 1px solid #ccc;
  border-radius: 4px;
  padding: 4px;
  z-index: 1001;
  box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
}

#more-dropdown.hidden {
  display: none;
}

#more-dropdown button {
  margin: 2px 0;
  width: 100%;
  text-align: left;
}

#searchBox {
  padding: 6px 10px;
  border: 1px solid #ccc;
  border-radius: 4px;
  font-size: 14px;
  min-width: 200px;
  background: white;
}

#time-slider-container {
  position: absolute;
  bottom: 100px;
  left: 50%;
  transform: translateX(-50%);
  z-index: 1000;
  text-align: center;
  background: rgba(255, 255, 255, 0.95);
  padding: 12px 20px;
  border-radius: 10px;
  box-shadow: 0 0 8px rgba(0, 0, 0, 0.25);
  width: 50%;
}

#time-slider {
  width: 100%;
  height: 20px;
  margin: 10px 0;
  accent-color: #0078D7; /* Modern browsers */
}

#time-label {
  font-size: 18px;
  font-weight: bold;
  display: block;
}



timeline-script.js?

ROUTE = /timeline-script.js


In [ ]:

const map = L.map('map', {
  maxBoundsViscosity: 1.0,
  //zoomSnap: 0.1, // allows fractional zooms like 5.3, 7.8, etc.
  //zoomDelta: 0.1 // optional, controls how much each zoom step changes
}).setView([20, 0], 2);

const worldBounds = L.latLngBounds(L.latLng(-180, -Infinity),
  L.latLng(180, Infinity));

const minZoom = map.getBoundsZoom(worldBounds, true); // true = inside
map.setMinZoom(minZoom);
map.setMaxBounds(worldBounds);
map.options.maxBoundsViscosity = 1.0;

//L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
//}).addTo(map);

const mainLayer = L.tileLayer('https://{s}.basemaps.cartocdn.com/rastertiles/voyager/{z}/{x}/{y}.png', {
  minZoom: 3,
  maxZoom: 17,
  attribution: '&copy; <a href="https://carto.com/">carto.com</a> contributors'
});
mainLayer.addTo(map);

let markerGroup = L.layerGroup().addTo(map);
let lineGroup = L.layerGroup().addTo(map);

function redrawMarkersAndLines() {
  markerGroup.clearLayers();
  lineGroup.clearLayers();

  const center = map.getCenter();
  const centerWrap = Math.round(center.lng / 360);
  const repeats = [centerWrap - 1,
    centerWrap,
  centerWrap + 1];

  repeats.forEach(r => {
    const latlngs = [];

    locations.forEach(loc => {
      const lngWrapped = loc.lng + (r * 360);
      const marker = L.marker([loc.lat, lngWrapped]);
      if (r === centerWrap) marker.bindPopup(loc.name); // Only bind popup on center
      marker.addTo(markerGroup);
      latlngs.push([loc.lat, lngWrapped]);
    });

    if (latlngs.length > 1) {
      L.polyline(latlngs, {
        color: 'blue'
      }).addTo(lineGroup);
    }
  })

}

let zooming = false
function updateBoundsRequest() {
  if (zooming) {
    return
  }
  const bounds = map.getBounds();
  const sw = bounds.getSouthWest(); // southwest corner
  const ne = bounds.getNorthEast(); // northeast corner
  const center = map.getCenter();
  const normalizedCenter = center.wrap();

  const viewBounds = [
    normalizedCenter.lng - Math.abs(ne.lng - sw.lng) / 2,
    normalizedCenter.lat - Math.abs(ne.lat - sw.lat) / 2,
    normalizedCenter.lng + Math.abs(ne.lng - sw.lng) / 2,
    normalizedCenter.lat + Math.abs(ne.lat - sw.lat) / 2,
  ]

  fetch('/countries.json?t=' + Date.now()
    + '&spanning[0]=' + sw.lng + '&spanning[1]=' + sw.lat
    + '&spanning[2]=' + ne.lng + '&spanning[3]=' + ne.lat
    + '&bounds[0]=' + viewBounds[0] + '&bounds[1]=' + viewBounds[1]
    + '&bounds[2]=' + viewBounds[2] + '&bounds[3]=' + viewBounds[3])
    .then(r => r.json()).then(results => {
      locations = results
      redrawMarkersAndLines()
    })
  //redrawMarkersAndLines()
}

updateBoundsRequest()


// Redraw when map finishes moving
map.on('moveend', () => {
  updateBoundsRequest();
  redrawMarkersAndLines();
})
// Initial draw
redrawMarkersAndLines();



category map controls?

ROUTE = /map-controls.js


In [ ]:
const toolbar = document.getElementById('category-toolbar');
const buttons = toolbar.querySelectorAll('button[data-category]');
const moreBtn = document.getElementById('more-button');
const dropdown = document.getElementById('more-dropdown');

function zoomToTightHorizontalBounds(map, targetSpan = 0.05, delay = 100) {
  function getSpan() {
    const b = map.getBounds();
    const lng1 = b.getWest();
    const lng2 = b.getEast();
    const horizontalSpan = Math.abs(lng2 - lng1) / 360;
    return horizontalSpan;
  }

  let zoomingOut = targetSpan > getSpan()
  function animateZoom() {
    zooming = true
    const currentSpan = getSpan();
    if ((!zoomingOut && currentSpan < targetSpan)
      || (zoomingOut && currentSpan > targetSpan)
    ) {
      zooming = false
      console.log("Reached target horizontal span:", currentSpan);
      return;
    }

    const center = map.getCenter();
    const nextZoom = map.getZoom() + (zoomingOut ? -1 : 1);
    map.setView(center, nextZoom, { animate: true });

    // Wait for animation to finish, then zoom again
    setTimeout(animateZoom, delay);
  }

  animateZoom();
}


buttons.forEach(button => {
  button.addEventListener('click', () => {
    const category = button.getAttribute('data-category');
    console.log('Switching to category:', category);
    if (category === 'cities') {
      zoomToTightHorizontalBounds(map)
    } else if (category === 'countries') {
      zoomToTightHorizontalBounds(map, 0.5)
    } else if (category === 'historic') {
      zoomToTightHorizontalBounds(map, 0.02)
    }
    // You can call your backend fetch/render logic here
  });
});

moreBtn.addEventListener('click', () => {
  dropdown.classList.toggle('hidden');
});

// Optional: auto-close dropdown when clicking elsewhere
document.addEventListener('click', (e) => {
  if (!toolbar.contains(e.target)) {
    dropdown.classList.add('hidden');
  }
});

function getBoundsFromPlaces(places) {
  let minLat = Infinity, maxLat = -Infinity;
  let minLng = Infinity, maxLng = -Infinity;

  for (let i = 0; i < places.length; i++) {
    const lat = places[i].lat;
    const lng = places[i].lng;

    if (lat < minLat) minLat = lat;
    if (lat > maxLat) maxLat = lat;
    if (lng < minLng) minLng = lng;
    if (lng > maxLng) maxLng = lng;
  }

  return [[minLat, minLng], [maxLat, maxLng]]; // Leaflet format
}


document.getElementById('searchBox').addEventListener('keypress', async function (e) {
  if (e.key === 'Enter') {
    const query = e.target.value.trim();
    if (query.length > 0) {
      // Do something with the query
      let response = await fetch('/countries.json?t=' + Date.now()
        + '&places=' + query)
      let data = await response.json()
      if (data && data.length) {
        const bounds = getBoundsFromPlaces(data);
        zooming = true
        setTimeout(() => zooming = false, 1000)
        map.fitBounds(bounds, {
          padding: [20, 20], // Optional padding
          animate: true      // Smooth animation
        });
        locations = data
        redrawMarkersAndLines()

      }
    }
  }
});

const slider = document.getElementById('time-slider');
const label = document.getElementById('time-label');

// Set the max to the current year
slider.min = 1800;
slider.max = 1900;
slider.value = slider.max;
label.textContent = `Year: ${slider.value}`;

slider.addEventListener('input', () => {
  label.textContent = `Year: ${slider.value}`;
  // TODO: update map markers or backend query based on the selected year
  // Example: updateHistoricalLayer(slider.value);
});




occupations.json?


In [ ]:
[
  {
    "armyName": "British Army",
    "year": 1801,
    "originalCountry": "United Kingdom",
    "occupationCountry": "Egypt"
  },
  {
    "armyName": "Napoleonic French Army",
    "year": 1807,
    "originalCountry": "France",
    "occupationCountry": "Spain"
  },
  {
    "armyName": "Russian Imperial Army",
    "year": 1812,
    "originalCountry": "Russia",
    "occupationCountry": "Poland"
  },
  {
    "armyName": "British Army",
    "year": 1815,
    "originalCountry": "United Kingdom",
    "occupationCountry": "South Africa"
  },
  {
    "armyName": "Ottoman Army",
    "year": 1821,
    "originalCountry": "Ottoman Empire",
    "occupationCountry": "Greece"
  },
  {
    "armyName": "French Army",
    "year": 1830,
    "originalCountry": "France",
    "occupationCountry": "Algeria"
  },
  {
    "armyName": "British East India Company",
    "year": 1843,
    "originalCountry": "United Kingdom",
    "occupationCountry": "Sindh"
  },
  {
    "armyName": "United States Army",
    "year": 1846,
    "originalCountry": "United States",
    "occupationCountry": "California"
  },
  {
    "armyName": "Austrian Imperial Army",
    "year": 1849,
    "originalCountry": "Austria",
    "occupationCountry": "Hungary"
  },
  {
    "armyName": "French Army",
    "year": 1853,
    "originalCountry": "France",
    "occupationCountry": "Crimea"
  },
  {
    "armyName": "British Army",
    "year": 1858,
    "originalCountry": "United Kingdom",
    "occupationCountry": "India"
  },
  {
    "armyName": "Russian Imperial Army",
    "year": 1864,
    "originalCountry": "Russia",
    "occupationCountry": "Caucasus"
  },
  {
    "armyName": "United States Army",
    "year": 1865,
    "originalCountry": "United States",
    "occupationCountry": "Southern U.S."
  },
  {
    "armyName": "French Army",
    "year": 1867,
    "originalCountry": "France",
    "occupationCountry": "Mexico"
  },
  {
    "armyName": "Prussian Army",
    "year": 1871,
    "originalCountry": "Germany",
    "occupationCountry": "Alsace-Lorraine"
  },
  {
    "armyName": "British Army",
    "year": 1878,
    "originalCountry": "United Kingdom",
    "occupationCountry": "Cyprus"
  },
  {
    "armyName": "Russian Imperial Army",
    "year": 1878,
    "originalCountry": "Russia",
    "occupationCountry": "Bulgaria"
  },
  {
    "armyName": "German Army",
    "year": 1884,
    "originalCountry": "Germany",
    "occupationCountry": "Namibia"
  },
  {
    "armyName": "Belgian Army",
    "year": 1885,
    "originalCountry": "Belgium",
    "occupationCountry": "Congo"
  },
  {
    "armyName": "Italian Army",
    "year": 1890,
    "originalCountry": "Italy",
    "occupationCountry": "Eritrea"
  },
  {
    "armyName": "British Army",
    "year": 1898,
    "originalCountry": "United Kingdom",
    "occupationCountry": "Sudan"
  }
]



lat/lon provider?

ROUTE = /countries.json


In [ ]:
const fs = require("fs");
const path = require("path");
const Papa = require("papaparse");

const TIMELINES = path.join(path.dirname(__dirname), "Resources", "Projects", "timelines");
const CACHE = {};
const LOCATIONS = {
  "countries.csv": ["abbrv", "lat", "lng", "country"],
  "worldcities.csv": true,
  "places.csv": true,
};

function geolocationProvider(places, bounds, spanning) {
  places = (places || "").split(",").map(c => c.trim().toLowerCase());
  const isPlacesEmpty = places.length === 0 || (places.length === 1 && places[0].trim() === "");
  const year = new Date().getFullYear();

  let isTooBig = true;
  let isStillTooBig = true;

  if (bounds) {
    //bounds = normalizeBoundsToVisibleWorld(bounds);
    const horizontalSpan = (bounds[2] - bounds[0]) / 360;

    if (horizontalSpan < 0.05) {
      isTooBig = false;
      console.log("Not too big.");
    }
    if (horizontalSpan < 0.02) {
      isStillTooBig = false;
      console.log("Not still too big.");
    }

    console.log(horizontalSpan, bounds, spanning);
  }

  let allPlaces = [];
  const timelines = Object.keys(LOCATIONS);

  for (let i = 0; i < timelines.length; i++) {
    const results = loadCSV(path.join(TIMELINES, timelines[i]))
      .filter(p => {
        //console.log(p)
        return isPlacesEmpty
          || (p.title && places.includes((p.title + '').toLowerCase()))
          || (p.city_ascii && places.includes((p.city_ascii + '').toLowerCase()))
          || (p.admin_name && places.includes((p.admin_name + '').toLowerCase()))
          || (p.country && places.includes((p.country + '').toLowerCase()))
      })
      .map(p => ({
        name: p.title || (p.city_ascii ? (p.city_ascii + ', ' + p.admin_name) : void 0) || p.country,
        lat: p.representative_latitude || p.lat,
        lng: p.representative_longitude || p.lng,
        year: year,
      }))
      .filter(row => row.lat && row.lng)
      .filter(p => !bounds || placeWithinBounds(p, bounds))

    allPlaces = allPlaces.concat(results);

    if (isTooBig && isPlacesEmpty) break;
    if (i === 1 && isStillTooBig && isPlacesEmpty) break;
  }

  return allPlaces;
}

function placeWithinBounds(p, bounds) {
  const lat = p.lat;
  const lng = p.lng;

  const minLat = parseFloat(bounds[1]);
  const maxLat = parseFloat(bounds[3]);
  const minLng = parseFloat(bounds[0]);
  const maxLng = parseFloat(bounds[2]);

  const latOk = lat >= minLat && lat <= maxLat;

  if (minLng < -180 && latOk && lng >= minLng + 360) {
    return true
  }

  if (maxLng > 180 && latOk && lng <= maxLng - 360) {
    return true
  }

  // If longitude range doesn't wrap (normal case)
  if (minLng <= maxLng) {
    return latOk && lng >= minLng && lng <= maxLng;
  }

  // If longitude range wraps (e.g. from 170 to -170)
  return latOk && (lng >= minLng || lng <= maxLng);
}

function loadCSV(filePath) {
  if (CACHE[filePath]) {
    return CACHE[filePath];
  }

  const csvData = fs.readFileSync(filePath, "utf8");
  const headers = LOCATIONS[path.basename(filePath)];

  const parsed = Papa.parse(csvData, {
    header: !!headers,
    skipEmptyLines: true,
    dynamicTyping: true,
    transformHeader: (name, index) => headers === true ? name : headers[index]
  });

  CACHE[filePath] = parsed.data;
  return parsed.data;
}

module.exports = geolocationProvider;
